<a href="https://colab.research.google.com/github/kevindinh24/BERT-Fine-Tuning-on-GLUE/blob/main/BERT_on_mnli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets
!pip install torch

In [ ]:
from datasets import load_dataset
dataset = load_dataset("glue", "mnli")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from transformers import AutoTokenizer #Loads pre trained BERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
def tokenize_function(examples):
  return tokenizer(examples["premise"], examples["hypothesis"], truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched = True)

Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer) #Pads input to be same length to save memory

In [ ]:
%env WANDB_DISABLED=true

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

#load model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=3)

#training arguemnts
training_args = TrainingArguments("test_trainer")

#Create trainer instance
trainer = Trainer(
    model=model,
    args = training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation_matched"],
    data_collator = data_collator,
    tokenizer = tokenizer
)

trainer.train()

env: WANDB_DISABLED=true


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-13910590.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,1.009500
1000,0.848500
1500,0.798400
2000,0.775600
2500,0.784100
3000,0.780100
3500,0.769800
4000,1.088000
4500,1.113600
5000,1.100500


KeyboardInterrupt: 

In [ ]:
# MNLI has two validation sets: "validation_matched" and "validation_mismatched"
predictions = trainer.predict(tokenized_datasets["validation_matched"])
pred_labels = np.argmax(predictions.predictions, axis=1)
true_labels = predictions.label_ids

import evaluate
metric = evaluate.load("glue", "mnli")

result = metric.compute(predictions=pred_labels, references=true_labels)
print("Validation Matched:", result)

# Do the same for validation_mismatched
predictions_mm = trainer.predict(tokenized_datasets["validation_mismatched"])
pred_labels_mm = np.argmax(predictions_mm.predictions, axis=1)
true_labels_mm = predictions_mm.label_ids

result_mm = metric.compute(predictions=pred_labels_mm, references=true_labels_mm)
print("Validation Mismatched:", result_mm)

Step,Training Loss
500,1.009500
1000,0.848500
1500,0.798400
2000,0.775600
2500,0.784100
3000,0.780100
3500,0.769800
4000,1.088000
4500,1.113600
5000,1.100500


Validation Matched: {'accuracy': 0.7348955680081508}
Validation Mismatched: {'accuracy': 0.7449145646867372}


In [ ]:
!rm -rf ~/.cache/huggingface